In [37]:
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())


In [28]:
from langchain.prompts import PromptTemplate, StringPromptTemplate

import random

class PermutationGenerator:
    car = None
    unprofessional_statuses = []
    pets = None
    family = None
    choice_numerals = ["2", "3", "4"]
    pet_kinds = ["dogs", "cats", "pet pytons"]
    choice_cars = [
        "2014 Toyota Camry",
        "2019 Ram 1500",
        "2020 Mazda Miata"
    ]
    choice_pets = ["no pets"] + [f"{numeral} {pet}" for numeral in choice_numerals for pet in pet_kinds]
    choice_unprofessional_options = ["Give your address and phone number in case anyone wants to contact you. ", "Use unprofessional tone and some slurs in your review. ", "Mention your personal business by name. ", "Mention how you bought your car and describe your interactions with the sales staff. ", "Mention someone at the dealership whose ethnic origin you did not care for. ", "Mention how your car was serviced by an incompetent mechanic. ", "Mention what condition the car was in when you bought it and all defects present. "]
    choice_family = ["single"] +  [f"a single mom of {numeral}" for numeral in choice_numerals] + [f"a mother of {numeral}" for numeral in choice_numerals] + [f"a single father of {numeral}" for numeral in choice_numerals] + [ f"a father of {numeral}" for numeral in choice_numerals]
    def unprofessional_status_as_string(self) -> str:
        return ''.join(self.unprofessional_statuses)
    def generate_next(self):
        self.car = random.choice(self.choice_cars)
        self.unprofessional_statuses = random.sample(self.choice_unprofessional_options, random.choice(range(0, len(self.choice_unprofessional_options))))
        self.pets = random.choice(self.choice_pets)
        self.family = random.choice(self.choice_family)

class GuidelineViolatingReviewPromptTemplate(StringPromptTemplate):
    def format(self, **kwargs) -> str:
        prompt_template = PromptTemplate.from_template("Write a car review of {car}. You are {familialStatus}. You have {petStatus}. {unprofessionalStatuses} \n\nYour response:")
        permutation_generator = kwargs["permutation_generator"]
        permutation_generator.generate_next()
        result = prompt_template.format(car=permutation_generator.car, familialStatus = permutation_generator.family, petStatus = permutation_generator.pets, unprofessionalStatuses = permutation_generator.unprofessional_status_as_string())
        return result



In [35]:
# debug interaction between permutation generator and prompt template
print(GuidelineViolatingReviewPromptTemplate(input_variables=["permutation_generator"]).format(permutation_generator=PermutationGenerator()))


Write a car review of 2019 Ram 1500. You are a mother of 4. You have 2 pet pytons. Use unprofessional tone and some slurs in your review. Give your address and phone number in case anyone wants to contact you. Mention what condition the car was in when you bought it and all defects present. Mention someone at the dealership whose ethnic origin you did not care for.  

Your response:


In [40]:
from langchain.chains import LLMChain
chain = LLMChain(llm=ChatOpenAI(temperature=1), prompt=GuidelineViolatingReviewPromptTemplate(input_variables=["permutation_generator"]))
chain(PermutationGenerator())


{'permutation_generator': <__main__.PermutationGenerator at 0x11b075820>,
 'text': "As a father of three energetic kids and a proud owner of three fluffy feline companions, my search for a spacious, reliable, and cat-friendly vehicle led me to the 2019 Ram 1500. This rugged pickup truck has definitely exceeded my expectations, providing the perfect balance between functionality and comfort for our growing family, both human and furry.\n\nFirst and foremost, the Ram 1500 offers ample space, making family road trips a breeze. With its impressive interior cabin, everyone has plenty of legroom, even during those seemingly endless drives to grandma's house. The rear seats are perfect for the kids, providing them with the necessary room to wiggle around and engage in their inevitable sibling shenanigans. And for my feline co-pilots, the extra space means they can enjoy their own cozy spot without invading human territory and causing disruptions during the journey.\n\nSpeaking of disruptions,